In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.applications import EfficientNetV2B0
from keras import regularizers

In [ ]:
img_size = (224, 224)

# train data
cwd = os.getcwd()
epochs = 15
df = pd.read_csv(cwd + '/augmentation_data.csv')
X = list(df['image'][i] for i in range(len(df)))
y = list(df['emotion'][i] for i in range(len(df)))

In [ ]:
weight = dict(enumerate(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)))
print(weight)

In [ ]:
def create_model():
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(img_size[0], img_size[1], 3)),
        layers.experimental.preprocessing.RandomRotation(
            0.05, fill_mode='nearest'
        ),

        EfficientNetV2B0(
            include_top=False, weights='imagenet', input_tensor=None,
            input_shape=(img_size[0], img_size[1], 3), include_preprocessing=True
        ),

        tf.keras.layers.GlobalMaxPooling2D(),
        tf.keras.layers.Flatten(),


        tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(l=0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(l=0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l=0.01)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(8, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(amsgrad=True),  # tf.keras.optimizers.Adam(amsgrad=True),  # tf.keras.optimizers.SGD(nesterov=True), # tfa.optimizers.RectifiedAdam(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(
                      from_logits=True),
                  metrics=['accuracy'])

    return model

In [ ]:
# es_callback = tf.keras.callbacks.EarlyStopping(
#     monitor='val_accuracy', patience=epochs*0.05, restore_best_weights=True)
# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath="model/cp-{epoch:04d}.ckpt",
#     verbose=0,
#     save_weights_only=True,
#     monitor='val_accuracy',
#     save_best_only=True)
# tb_callback = tf.keras.callbacks.TensorBoard(
#     log_dir="./logs",
#             update_freq="epoch")

In [ ]:
model = create_model()
print(model.summary())


In [ ]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
# adam=Adam(lr=0.001)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['acc'])


In [ ]:
import os, cv2, shutil
df_train = pd.read_csv(cwd + "\\state\\train.csv")
df_test = pd.read_csv(cwd + "\\state\\test.csv")
df_val = pd.read_csv(cwd + "\\state\\val.csv")
print(df_val)
output_train_dir = cwd + "\\train"
#output_test_dir = cwd + "\\test"
root_image_dir = cwd + "\\augmentation\\"
dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
dir_test = cwd + "\\test\\"
dir_train = cwd + "\\train\\"
dir_val = cwd + "\\val\\"
if(os.path.exists(dir_train) or os.path.exists(dir_test) or os.path.exists(dir_val)):
    shutil.rmtree(dir_val)
    shutil.rmtree(dir_train)
    shutil.rmtree(dir_test)
for emotion in dictionary:
    os.makedirs(dir_val + "\\" + emotion)
    os.makedirs(dir_test + "\\" + emotion)
    os.makedirs(dir_train + "\\" + emotion)

def isInDir(image_dir):
    return len(image_dir.split("/")) >= 2
for index, row in df_val.iterrows():
    if(isInDir(row['image']) == False): 
        output_val_dir = dir_val + dictionary[row['emotion']] + "\\" + row['image']
    else:
        output_val_dir = dir_val + row['image']
    image = cv2.imread(root_image_dir + row['image'])
    cv2.imwrite(output_val_dir, image)
for index, row in df_train.iterrows():
    if(isInDir(row['image']) == False):
        output_train_dir = dir_train + dictionary[row['emotion']] + "\\" + row['image']
    else:
        output_train_dir = dir_train + row['image']
    image = cv2.imread(root_image_dir + row['image'])
    cv2.imwrite(output_train_dir, image)
for index, row in df_test.iterrows():
    if(isInDir(row['image']) == False): 
        output_test_dir = dir_test + dictionary[row['emotion']] + "\\" + row['image']
    else:
        output_test_dir = dir_test + row['image']
    image = cv2.imread(root_image_dir + row['image'])
    cv2.imwrite(output_test_dir, image)


In [ ]:
bs = 75
train_dir = cwd + "\\train"
test_dir = cwd + "\\test"
val_dir = cwd + "\\val"
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )
val_datagen = ImageDataGenerator( rescale = 1.0/255. )
train_generator=train_datagen.flow_from_directory(train_dir,batch_size=bs,class_mode='categorical',target_size=(img_size[0],img_size[1]))
test_generator=train_datagen.flow_from_directory(test_dir,batch_size=bs,class_mode='categorical',target_size=(img_size[0],img_size[1]))
validation_generator =  val_datagen.flow_from_directory(val_dir,
                                                         batch_size=bs,
                                                         class_mode  = 'categorical',
                                                         target_size=(img_size[0],img_size[1]))
dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    class_names=dictionary,
    color_mode='rgb',
    batch_size=bs)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    shuffle=True,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    class_names=dictionary,
    color_mode='rgb',
    batch_size=bs)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='int',
    image_size=img_size,
    class_names=dictionary,
    color_mode='rgb',
    batch_size=bs)

filenames = validation_generator.filenames
nb_samples = len(filenames)

In [ ]:
'''
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=epochs*0.1, restore_best_weights=True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model/cp-{epoch:04d}.ckpt",
    verbose=0,
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=True)
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir="./logs",
            update_freq="epoch")
'''

In [ ]:
history = model.fit_generator(train_ds, validation_data=val_ds, shuffle=True, epochs=epochs, #callbacks=[es_callback, cp_callback, tb_callback
                                                                                                   #, reduce_lr
                                                                                                  #],
                                                                                                  class_weight=weight, batch_size=bs)

In [ ]:
import matplotlib.pyplot as plt
#Plotting the training and validation loss

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'y', label='Training accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:

print(train_ds)
print('---------------')
print(val_ds)
print('---------------')


In [ ]:
cwd = os.getcwd()
n_df = pd.read_csv(cwd + "/state/val.csv")

In [ ]:
n_df

In [ ]:
for idx, row in df.iterrows():
    imagePath = cwd + "/cleaned_images/" + row.image
    image = cv2.imread(imagePath)

emo_feature = list(row.emotion for idx, row in df.iterrows())

In [ ]:
'''
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import auc

dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']

for idx in range(len(dictionary)):
    print(dictionary[idx])

    emo_feature = np.copy(X_train)
    emo_target = list(label[idx] for label in y_train)
    emo_target = np.array(emo_target)

    X = emo_feature
    y = emo_target

    history = []
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    fig, ax = plt.subplots()

    # plot ROC curves
    cv = StratifiedKFold(n_splits=5, shuffle=True)
    for i, (train, val) in enumerate(cv.split(X, y)):
        X_train_kf, X_val_kf = X[train], X[val]
        y_train_kf, y_val_kf = y[train], y[val]

        le = preprocessing.LabelEncoder()
        y_train_kf = to_categorical(y_train_kf)
        y_val_kf = to_categorical(y_val_kf)

        model = create_model_best_param(best_params, 2)
        model.fit(X_train_kf,
                    y_train_kf,
                    validation_data=(X_val_kf, y_val_kf,),
                    batch_size=best_params['batch_size'],
                    epochs=best_params["epochs"],
                    verbose=2)

        # predict
        y_pred = model.predict(X_val_kf).ravel()
        y_val_kf = y_val_kf.ravel()

        print('====================Fold ', i , '====================')

        # plot ROC curve
        viz = RocCurveDisplay.from_predictions(y_val_kf, y_pred, ax=ax, name="ROC fold {}".format(i), alpha=0.3, lw=1,)
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    # middle line
    ax.plot([0, 1], [0, 1], 'k--')

    # mean line
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )

    # std
    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(xlim=[-0.05, 1.05],
            ylim=[-0.05, 1.05],
            title="Receiver operating characteristic")
    ax.legend(loc="lower right")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.savefig(cwd + '/../graph/' + dictionary[idx] + '/ann_relu.jpg')
    plt.show()
'''

In [ ]:
# test model
# Confution Matrix and Classification Report

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']

Y_pred = model.predict(test_ds)
y_pred = np.argmax(Y_pred, axis=1)
true_categories = tf.concat([y for x, y in test_ds], axis=0)
print('Confusion Matrix')
cm = confusion_matrix(true_categories, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=dictionary)
fig, ax = plt.subplots(figsize=(10,10))
disp.plot(ax=ax)
#plt.savefig('evaluate_matrix_embedded_2/gnb_f.png',backend=None)
plt.show()

print(classification_report(test_ds, y_pred))
